In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install git+https://github.com/beroguedou/SpecAugment.git
#!pip install torchaudio

In [3]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
from decode import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim
from transformers import BertTokenizer

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed_value = 2020
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

In [4]:

limit = 80
params = {'batch_size': None,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


In [5]:
%%time 

training_set = LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')
dev_set = LibriSpeechDataset(limit=limit, n_frames=600, version='dev-clean')


CPU times: user 375 ms, sys: 107 ms, total: 482 ms
Wall time: 1.46 s


In [6]:
# Let's see a single utterance

LibriSpeechDataset(limit=limit, n_frames=600, version='train-clean-360')[0]

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor([  101,   101,  2045,  2001,  2625,  1997,  1037,  4306,  2055,  2085,
          1998, 14163,  6894,  5289,  2106,  2025,  2031,  2000,  2562,  2067,
          1037,  5481,  2004,  2002,  2441,  1996,  9445,  3460,  6031,  2988,
          1996, 14460,  4214,  8721,  2000, 10767,  1998, 27468,  1996,  2221]))

In [7]:
encoder = EncoderCONV2DRNN(device=device, hidden_size=64).to(device)

decoder = DecoderATTRNN(30000, dec_units=64, hidden_size=64).to(device)


encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [26]:
# Train the model

global_trainer(5, training_set, dev_set, params, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device)

 ====== ====== ====== ====== ====== ======
      The model has 4954139 parameters
 ====== ====== ====== ====== ====== ======


Epoch        5: 100%|████████████████████| 5/5 [01:12<00:00, 14.46s/it, Train loss 5.0485 Eval loss 6.2242]

Time taken for the training 0.10158 hours



In [ ]:
mfccs, references = training_set[1]
tokenizer =  BertTokenizer.from_pretrained('bert-base-uncased')
references = [tokenizer.convert_ids_to_tokens(ind) for ind in references.numpy().tolist()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang_tokenizer=tokenizer, 
          device=device, beam_search=True)

In [ ]:
# Vérifications

# 1 - Data Augmentation
# 2 - Encoder
# 3 - Attention Mechanism Bahdanau Audio
# 4 - Smoothing and Topk to the attention
# 5 - Decoder 
# 6 - Métrique BLEU